In [ ]:
import pandas as pd
import json

In [ ]:
with open('action.log', 'r') as f:
    action_logs = json.load(f)

with open('emotion.log', 'r') as f:
    emotion_logs = json.load(f)

In [ ]:
pivot = 0
action_timestamps = list(action_logs.keys())
emotion_timestamps = list(emotion_logs.keys())

start = ''
for i, emotion_timestamp in enumerate(emotion_timestamps):
    if action_timestamps[0] <= emotion_timestamp:
        start = i
        break

for emotion_timestamp in emotion_timestamps[start:]:
    for k, action_timestamp in enumerate(action_timestamps[pivot:]):
        if abs(int(action_timestamp)-int(emotion_timestamp)) < 15:
            osizes = []
            fsizes = []

            action_log = action_logs[action_timestamp] # {obj_id: {'pos':..., 'actions':[...]}, obj_id: ...}
            emotion_log = emotion_logs[emotion_timestamp] # [{'pos':..., 'emotions':[...]}, ...]

            for obj_id, info in action_log.items():
                l, t, r, b = info['pos']
                size = (r-l)*(b-t)
                osizes.append((obj_id, size))

            for j, face in enumerate(emotion_log):
                l, t, r, b = face['pos']
                size = (r-l)*(b-t)
                fsizes.append((j, size))

            osizes.sort(key=lambda x:x[1])
            fsizes.sort(key=lambda x:x[1])

            row_nums = len(osizes)
            col_nums = len(fsizes)

            matrix = [[0]*col_nums for _ in range(row_nums)]

            for i in range(row_nums):
                l, t, r, b = action_log[osizes[i][0]]['pos']

                for j in range(col_nums):
                    fsize = fsizes[j] # (idx, size)
                    fl, ft, fr, fb = emotion_log[fsize[0]]['pos']

                    # overlapped grid
                    ovl = max(l, fl)
                    ovt = max(t, ft)
                    ovr = min(r, fr)
                    ovb = min(b, fb)

                    overlapped_size = (ovr-ovl)*(ovb-ovt)

                    matrix[i][j] = overlapped_size/fsize[1]
            
            for j in range(col_nums):
                col = [matrix[i][j] for i in range(row_nums)]
                overlapped_size_max = max(col)

                for i, overlapped_size in enumerate(col):
                    if overlapped_size == overlapped_size_max and j <= i:
                        emotion_log[fsizes[j][0]]['id'] = osizes[i][0]
                        break

            pivot += k
            break